In [1]:
#Inspiration :D
#https://gitlab.com/ryanlogsdon/pubsub-for-python/-/blob/main/subscriber.py
#https://www.youtube.com/watch?v=xOtrCmPjal8&ab_channel=D-I-Ry
import os
from google.cloud import pubsub_v1
import sys
import time
import pandas as pd
import http.client
import numpy as np
import datetime
from io import StringIO
import functools as ft

In [2]:
! python --version

Python 3.7.4


## Create connection to PubSub

In [3]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [4]:
publisher_obs = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

publisher_forecast = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

topic_path_obs = 'projects/weather-based-forecasting-v2/topics/observations_topic'
topic_path_forecast = 'projects/weather-based-forecasting-v2/topics/forecasts_topic'


## Download the data

In [5]:
time.sleep(5)

locations = ["Berlin", "Hamburg","Bremen"]


conn = http.client.HTTPSConnection("visual-crossing-weather.p.rapidapi.com")

headers = {
    "X-RapidAPI-Key": "c7b48b7c4emsh8a3b688e71b9896p1be4d0jsnda2e5b27b486",
    "X-RapidAPI-Host": "visual-crossing-weather.p.rapidapi.com"
}

df_list = []
for location in locations:

    conn.request("GET", "/forecast?location={}&aggregateHours=1&shortColumnNames=0&unitGroup=metric&contentType=csv".format(location),
                 headers=headers)

    res = conn.getresponse()
    data = res.read()
    s = str(data, 'utf-8')
    data_string = StringIO(s)
    df = pd.read_csv(data_string)
    # df.to_csv("./weather_prediction_{}.csv".format(location))
    df = df.add_suffix("_{}".format(location))
    df["Date time"] = df["Date time_{}".format(location)]
    df = df.drop(columns=["Date time_{}".format(location)])
    df_list.append(df)

In [6]:
df_merged = ft.reduce(lambda left, right: pd.merge(left, right, on='Date time'), df_list)
df_merged.head(5)

,Address_Berlin,Latitude_Berlin,Longitude_Berlin,Resolved Address_Berlin,Name_Berlin,Wind Direction_Berlin,Temperature_Berlin,Wind Speed_Berlin,Cloud Cover_Berlin,Heat Index_Berlin,...,Heat Index_Bremen,Chance Precipitation (%)_Bremen,Precipitation_Bremen,Sea Level Pressure_Bremen,Snow Depth_Bremen,Snow_Bremen,Relative Humidity_Bremen,Wind Gust_Bremen,Wind Chill_Bremen,Conditions_Bremen
0,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,237.8,5.6,27.0,95.3,NaN,...,NaN,71.4,0.0,999.7,0.0,0.0,87.5,62.8,-0.1,"Rain, Overcast"
1,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,226.3,5.7,26.6,34.2,NaN,...,NaN,19.0,0.0,999.3,0.0,0.0,88.8,56.0,-0.2,Overcast
2,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,226.2,5.8,27.4,48.2,NaN,...,NaN,19.0,0.0,999.0,0.0,0.0,81.6,49.3,0.4,Overcast
3,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,224.1,5.3,26.3,76.6,NaN,...,NaN,19.0,0.0,998.7,0.0,0.0,78.7,43.9,0.8,Overcast
4,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,227.9,4.9,26.6,70.7,NaN,...,NaN,19.0,0.0,998.3,0.0,0.0,81.5,38.5,0.8,Overcast


In [7]:
df2 = df_merged.copy()
df2.index = pd.to_datetime(df2["Date time"])
df2 = df2.asfreq(freq="1min")
df2 = df2.interpolate(method ='linear', limit_direction ='forward')

for location in locations:
    df2["Conditions_{}".format(location)] = df2["Conditions_{}".format(location)].fillna(method="backfill")
    df2["Address_{}".format(location)] = df2["Address_{}".format(location)][0]
    df2["Resolved Address_{}".format(location)] = df2["Name_{}".format(location)][0]
    df2["Name_{}".format(location)] = df2["Name_{}".format(location)][0]
df2["Date time"] = df2.index

# we're taking 10 days of a forecast
# now = datetime.datetime.now()
# minutes = now.minute
# df3 = df2.iloc[minutes-1:60*24*10, :]
df3 = df2.iloc[:60*24*10, :]
#df3.to_csv(sys.stdout, index=False)

In [8]:
now = datetime.datetime.now()
minutes = now.minute
df3_hour = df3.iloc[minutes-1:minutes+60, :]
df3_hour.head(5)

,Address_Berlin,Latitude_Berlin,Longitude_Berlin,Resolved Address_Berlin,Name_Berlin,Wind Direction_Berlin,Temperature_Berlin,Wind Speed_Berlin,Cloud Cover_Berlin,Heat Index_Berlin,...,Heat Index_Bremen,Chance Precipitation (%)_Bremen,Precipitation_Bremen,Sea Level Pressure_Bremen,Snow Depth_Bremen,Snow_Bremen,Relative Humidity_Bremen,Wind Gust_Bremen,Wind Chill_Bremen,Conditions_Bremen
Date time,,,,,,,,,,,,,,,,,,,,,
2023-01-15 19:19:00,Berlin,52.516,13.3769,Berlin,Berlin,234.158333,5.631667,26.873333,75.951667,NaN,...,NaN,54.806667,0.0,999.573333,0.0,0.0,87.911667,60.646667,-0.131667,Overcast
2023-01-15 19:20:00,Berlin,52.516,13.3769,Berlin,Berlin,233.966667,5.633333,26.866667,74.933333,NaN,...,NaN,53.933333,0.0,999.566667,0.0,0.0,87.933333,60.533333,-0.133333,Overcast
2023-01-15 19:21:00,Berlin,52.516,13.3769,Berlin,Berlin,233.775000,5.635000,26.860000,73.915000,NaN,...,NaN,53.060000,0.0,999.560000,0.0,0.0,87.955000,60.420000,-0.135000,Overcast
2023-01-15 19:22:00,Berlin,52.516,13.3769,Berlin,Berlin,233.583333,5.636667,26.853333,72.896667,NaN,...,NaN,52.186667,0.0,999.553333,0.0,0.0,87.976667,60.306667,-0.136667,Overcast
2023-01-15 19:23:00,Berlin,52.516,13.3769,Berlin,Berlin,233.391667,5.638333,26.846667,71.878333,NaN,...,NaN,51.313333,0.0,999.546667,0.0,0.0,87.998333,60.193333,-0.138333,Overcast


## Send observations in intervals to PubSub - check Weather_PubSub_check for results

In [10]:
for i in range(10):
    obs = str(df3_hour.iloc[i,:].astype(str).to_dict())
    obs = obs.replace("'", '"')
    index = df3_hour.iloc[i,:].name
    index = str(int(index.value / 10**9)).encode("utf-8")
    obs = obs.encode("utf-8")
    future = publisher_obs.publish(topic_path_obs, obs, ordering_key=index)
    print(f'published message id {future.result()}')
    time.sleep(6)

published message id 6729670383628395
published message id 6729670383622979
published message id 6729670487022392
published message id 6729670577495248
published message id 6729711686831930
published message id 6729670584744982
published message id 6729670506243700
published message id 6729670383589853
published message id 6729670584733105
published message id 6729712511471291


## Send Forecast in intervals to PubSub

In [13]:
df3_forecast = df3.iloc[minutes-1:, :]
df3_forecast.index = df3_forecast.index.astype(str)
df3_forecast.head(5)

,Address_Berlin,Latitude_Berlin,Longitude_Berlin,Resolved Address_Berlin,Name_Berlin,Wind Direction_Berlin,Temperature_Berlin,Wind Speed_Berlin,Cloud Cover_Berlin,Heat Index_Berlin,...,Heat Index_Bremen,Chance Precipitation (%)_Bremen,Precipitation_Bremen,Sea Level Pressure_Bremen,Snow Depth_Bremen,Snow_Bremen,Relative Humidity_Bremen,Wind Gust_Bremen,Wind Chill_Bremen,Conditions_Bremen
Date time,,,,,,,,,,,,,,,,,,,,,
2023-01-15 12:43:00,Berlin,52.516,13.3769,Berlin,Berlin,231.496667,7.271667,23.703333,100.0,NaN,...,NaN,23.8,0.028333,992.501667,0.0,0.0,69.508333,68.956667,0.643333,Overcast
2023-01-15 12:44:00,Berlin,52.516,13.3769,Berlin,Berlin,231.473333,7.273333,23.726667,100.0,NaN,...,NaN,23.8,0.026667,992.513333,0.0,0.0,69.466667,68.953333,0.646667,Overcast
2023-01-15 12:45:00,Berlin,52.516,13.3769,Berlin,Berlin,231.450000,7.275000,23.750000,100.0,NaN,...,NaN,23.8,0.025000,992.525000,0.0,0.0,69.425000,68.950000,0.650000,Overcast
2023-01-15 12:46:00,Berlin,52.516,13.3769,Berlin,Berlin,231.426667,7.276667,23.773333,100.0,NaN,...,NaN,23.8,0.023333,992.536667,0.0,0.0,69.383333,68.946667,0.653333,Overcast
2023-01-15 12:47:00,Berlin,52.516,13.3769,Berlin,Berlin,231.403333,7.278333,23.796667,100.0,NaN,...,NaN,23.8,0.021667,992.548333,0.0,0.0,69.341667,68.943333,0.656667,Overcast


In [ ]:
for i in range(10):
    
    df3_forecast_window = str(df3_forecast.iloc[i:i+3*24*60,:].astype(str).to_dict("index"))
#     for key in df3_forecast_window.keys():
#         df3_forecast_window[key] = str(df3_forecast_window[key])
    df3_forecast_window = df3_forecast_window.encode('utf-8')
        
    info = "Weather forecast for Hamburg, {}".format(df3_forecast.index[i])
    info = info.encode('utf-8')
    
    future = publisher_forecast.publish(topic_path_forecast, df3_forecast_window)
    print(f'published message id {future.result()}')
    time.sleep(60)

published message id 6729461731670847
published message id 6729461550037848
published message id 6729461799472888
published message id 6729465709637464
published message id 6729466476817391
